In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/all.csv')
df=df[["ESR1.log2.tpm" ,"PGR.log2.tpm" , "IL12RB2","HORMAD1","LEMD1","AGR3","GFRA1","ANTXRL",'NXNL2',"SPATA4",'RNF182',
      "TP53"  , "COL5A1", "GATA3",'LYST',
      "Danaher.Mast.cells" , "Act.CD4"  , "MDSC",'NK cells','Tem.CD8','CD8 T cells',"Danaher.Neutrophils","Danaher.NK.cells","Monocytic lineage" ,'Danaher.Exhausted.CD8','Danaher.CD8.T.cells','Cytotoxic lymphocytes','Danaher.Cytotoxic.cells',
      "Taxane.FinalScore","Swanton.PaclitaxelScore", "Taxane.MitosisScore" ,"Swanton.MitosisScore" ,
                                     'IDO1','TIDE.TAM.M2',"ESC.ssgsea.notnorm","ESC.ssgsea.norm"        
,"GGI.ssgsea.notnorm","resp.pCR"]]

In [ ]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
class SelectAtMostKBest(SelectKBest):
    def _check_params(self, X, y):
        if not (self.k == "all" or 0 <= self.k <= X.shape[1]):
            # set k to "all" (skip feature selection), if less than k features are available
            self.k = "all"

In [ ]:
AUC_rf=[]
for feature in range(X.shape[1]+1):
    # 用特定特征删除后的特征集
    #X_drop = X.drop(X.columns[feature], axis=1)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if feature < X.shape[1]:
        X_drop = X.drop(X.columns[feature], axis=1)
    else:
        X_drop = X
        
    scaler = StandardScaler()
    kbest = SelectAtMostKBest(score_func=f_classif)
    rf = RandomForestClassifier(random_state=1)
    pipe = Pipeline(steps=[('scaler', scaler), ('kbest', kbest), ('rf', rf)])
    # Parameter ranges
    param_grid = { 'kbest__k': range(1,X.shape[1]),
                    "rf__max_depth": [3, None],
                    "rf__n_estimators": [5, 10, 25, 50, 100],
                    "rf__max_features": [0.05, 0.1, 0.2, 0.5, 0.7],
                    "rf__min_samples_split": [2, 3, 6, 10, 12, 15]
                    }
    # Optimisation
    search = RandomizedSearchCV(pipe, param_grid, cv=skf, scoring='roc_auc', return_train_score=True, n_jobs=10, verbose=0, n_iter=1000, random_state=0)

    search.fit(X_drop,y)
    
    best_params = search.best_params_
    best_score = search.best_score_
    AUC_rf.append(best_score)
    
    print(best_params,best_score)

{'rf__n_estimators': 25, 'rf__min_samples_split': 12, 'rf__max_features': 0.05, 'rf__max_depth': None, 'kbest__k': 16} 0.854491341991342
{'rf__n_estimators': 10, 'rf__min_samples_split': 3, 'rf__max_features': 0.1, 'rf__max_depth': 3, 'kbest__k': 13} 0.8636363636363636
{'rf__n_estimators': 50, 'rf__min_samples_split': 12, 'rf__max_features': 0.2, 'rf__max_depth': None, 'kbest__k': 29} 0.8537878787878788
{'rf__n_estimators': 25, 'rf__min_samples_split': 2, 'rf__max_features': 0.1, 'rf__max_depth': None, 'kbest__k': 19} 0.8537878787878788
{'rf__n_estimators': 5, 'rf__min_samples_split': 6, 'rf__max_features': 0.1, 'rf__max_depth': None, 'kbest__k': 10} 0.8500000000000002
{'rf__n_estimators': 50, 'rf__min_samples_split': 6, 'rf__max_features': 0.05, 'rf__max_depth': None, 'kbest__k': 28} 0.8517857142857143
{'rf__n_estimators': 5, 'rf__min_samples_split': 6, 'rf__max_features': 0.05, 'rf__max_depth': None, 'kbest__k': 14} 0.8528138528138527
{'rf__n_estimators': 10, 'rf__min_samples_split':

In [ ]:
AUC_rf

[0.854491341991342,
 0.8636363636363636,
 0.8537878787878788,
 0.8537878787878788,
 0.8500000000000002,
 0.8517857142857143,
 0.8528138528138527,
 0.849025974025974,
 0.8574675324675326,
 0.8563311688311689,
 0.8553571428571429,
 0.8617965367965368,
 0.8582251082251082,
 0.8554653679653679,
 0.8614718614718615,
 0.8591450216450216,
 0.8579004329004327,
 0.8617965367965368,
 0.864069264069264,
 0.8505952380952382,
 0.8542478354978356,
 0.8498376623376623,
 0.8541666666666667,
 0.8487554112554111,
 0.8521645021645021,
 0.8610660173160174,
 0.8520021645021645,
 0.8494047619047619,
 0.8590367965367965,
 0.8590367965367965,
 0.8630952380952379,
 0.8630952380952379,
 0.855952380952381,
 0.8606601731601732,
 0.8668831168831168,
 0.8668831168831168,
 0.8581709956709958,
 0.8579004329004327]